Importing libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import re
from json.decoder import JSONDecodeError
import datetime

Reading CSV Files

In [ ]:
import import_ipynb
import funciones 

class Actor:
    def __init__(self):
        self.no_films=0
        self.name=""
        self.retorno=0
        self.promedio=0
    
    def set_no_films(self, value):
        self.no_films = value

    def set_name(self, value):
        self.name = value

    def set_retorno(self, value):
        self.retorno = value

    def set_promedio(self, value):
        self.promedio = value

    def get_no_films(self):
        return self.no_films

    def get_name(self):
        return self.name

    def get_retorno(self):
        return self.retorno

    def get_promedio(self):
        return self.promedio

    def to_string_actor_info(self):
        return "El actor "+self.name+" ha participado de "+str(self.no_films)+" cantidad de filmaciones, el mismo ha conseguido un retorno de "+ str(self.retorno)+ " con un promedio de "+str(self.promedio)+ " por filmación"

class Info_Film:
    def __init__(self):
        self.title = ""
        self.year_movie = ""
        self.popularity = ""

    def set_title(self, value):
        self.title = value
    
    def set_year_movie(self, value):
        self.year_movie = value
    
    def set_popularity(self, value):
        self.popularity = value
    
    def get_title(self):
        return self.title
    
    def get_year_movie(self):
        return self.year_movie
    
    def get_popularity(self):
        return self.popularity
    
    def to_string(self):
        return "La película "+self.title+" fue estrenada en el año "+self.year_movie+" con un score/popularidad de "+self.popularity
    

class Films:
    
    def __init__(self):
        self.movies_final = self.init_movies()
        self.credits_final = self.init_credits()

    def init_movies(self):
        movies_dataset = pd.read_csv('resources/movies_dataset.csv',  encoding='utf8')
        movies_df_english_idx = movies_dataset[movies_dataset["spoken_languages"].str.contains("English|Español") == False].index
        movies_pre = movies_dataset.drop(movies_df_english_idx)
        movies_pre.drop_duplicates(inplace=True)
        movies_final = funciones.format_dataset_movies(movies_pre)
        return movies_final
        
    def init_credits(self):
        credits_dataset = pd.read_csv('resources/credits.csv',  encoding='latin-1')
        credits_df_idx = credits_dataset[credits_dataset["id"].isnull()].index
        credits_pre = credits_dataset.drop(credits_df_idx)
        credits_pre.drop_duplicates(inplace=True)
        credits_final = funciones.format_dataset_credits(credits_pre)
        return credits_final

    def get_actor_by_credits(self, credits, target_name, search_id_result = []):
        target_key = "name"
        count = 0

        for index, row in credits.iterrows():
            search_name_result = []
            flag = False
            funciones.json_full_search(target_key, row['cast_json'], search_name_result)
            for name in search_name_result:
                search_director_name = []
                if(name == target_name):
                    funciones.json_job_search("job", "Director", target_name, row['crew_json'], search_director_name)
                    if len(search_director_name) > 0:
                        for director in search_director_name:
                            if (director == target_name):
                                flag = False;
                            else:
                                flag = True
                    else:
                        flag = True
            if flag == True:
                count = count + 1
                search_id_result.append(row['id'])

        return count
    
    def get_actor(self, actor_name):
        search_id_result = []
        actor = Actor()
        actor.set_no_films(self.get_actor_by_credits(self.credits_final, actor_name, search_id_result))
        actor.set_name(actor_name)
        actor_return = []
        for id in search_id_result:
            id_int = int(id)
            
            row = self.movies_final[self.movies_final['id'] == id_int]
            actor_return.append(row['retorno'].values[0])

        my_actor_value = pd.Series(actor_return)
        print(my_actor_value)
        actor.set_retorno(my_actor_value.sum())
        actor.set_promedio(my_actor_value.mean())
        return actor
    
    def cantidad_filmaciones_mes(self, month):
        numeric_month=funciones.get_month_numeric(month)
        str_var = self.movies_final.groupby(['month_movies'])['month_movies'].count()[numeric_month]
        return str_var
        
    
   
    def cantidad_filmaciones_dia(self, day):
        day_formatted = day.lower()
        print(day_formatted)
        return self.movies_final.groupby(['day_movies'])['day_movies'].count()[day_formatted]
        

    def score_titulo(self, film_title):
        title = film_title.lower()
        
        info = Info_Film()
        info.set_title(film_title)
        info.set_popularity(str(self.movies_final[self.movies_final['key_title'] == title]['popularity'].values[0]))
        info.set_year_movie(str(self.movies_final[self.movies_final['key_title'] == title]['year_movies'].values[0]))

        return info
        
    #falta también
    #def votos_titulo( titulo_de_la_filmación ):

    def get_director(self, director_name):
        print("Peliculas para el mes de Enero sonEnero")
    
        